# Visualization: Trading Session

In [2]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [33]:
from trading_bot.agent import Agent

model_name = 'model_double-dqn_GOOG_50'
test_stock = 'data/TCS_GBR.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [35]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2020-08-24,2292.014532
1,2020-08-25,2293.110214
2,2020-08-26,2294.912203
3,2020-08-27,2283.826920
4,2020-08-28,2294.463707


### 3. Running Eval

In [36]:
import logging
# import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

# coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

### 4. Visualize

In [37]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [38]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [20]:
history

[(1045.849976, 'BUY'),
 (1016.0599980000001, 'BUY'),
 (1070.709961, 'SELL'),
 (1068.390015, 'BUY'),
 (1076.280029, 'SELL'),
 (1074.660034, 'SELL'),
 (1070.329956, 'HOLD'),
 (1057.189941, 'BUY'),
 (1044.689941, 'BUY'),
 (1077.150024, 'BUY'),
 (1080.969971, 'SELL'),
 (1089.900024, 'BUY'),
 (1098.26001, 'SELL'),
 (1070.52002, 'BUY'),
 (1075.569946, 'BUY'),
 (1073.900024, 'BUY'),
 (1090.98999, 'SELL'),
 (1070.079956, 'SELL'),
 (1060.619995, 'SELL'),
 (1089.060059, 'SELL'),
 (1116.369995, 'SELL'),
 (1110.75, 'BUY'),
 (1132.800049, 'SELL'),
 (1145.98999, 'HOLD'),
 (1115.22998, 'BUY'),
 (1098.709961, 'SELL'),
 (1095.060059, 'HOLD'),
 (1095.01001, 'BUY'),
 (1121.369995, 'BUY'),
 (1120.160034, 'BUY'),
 (1121.670044, 'BUY'),
 (1113.650024, 'BUY'),
 (1118.560059, 'SELL'),
 (1113.800049, 'BUY'),
 (1096.969971, 'BUY'),
 (1110.369995, 'BUY'),
 (1109.400024, 'SELL'),
 (1115.130005, 'SELL'),
 (1116.050049, 'SELL'),
 (1119.920044, 'SELL'),
 (1140.98999, 'SELL'),
 (1147.800049, 'SELL'),
 (1162.030029, '